In [2]:
import pandas as pd
from pathlib import Path
import json
import yaml

In [13]:
folder_to_harvest = Path("../data/douhet")

json_paths = folder_to_harvest.glob("**/*.json")
json_paths_with_labels = zip(map(lambda x: x.stem.replace("_data", ""), json_paths), json_paths)


db = pd.DataFrame()

for data_label, data_path in json_paths_with_labels:

    course_label = label.split("_")[0]
    cours_edition_label = label.split("_")[1]

    associazione_path = Path(f"./data/{course_label}/{course_label}_{cours_edition_label}/associazione.xlsx")
    associazione_df = pd.read_excel(associazione_path)
    
    with open(data_path, "r") as fin:
        
        data = json.loads(fin.read())
        root_key = list(data.keys())[0]
        
        sna_micro_stats_a = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_stats_b = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_sociogram = data[root_key]["sociogram"]["micro_stats"]
        
        sna_micro_stats_a_df = pd.DataFrame(sna_micro_stats_a).T.add_suffix("_a", axis=1)
        sna_macro_stats_b_df = pd.DataFrame(sna_micro_stats_b).T.add_suffix("_b", axis=1)
        sociogram_micro_stats_df = pd.DataFrame(sna_micro_sociogram).T
        
        course_df =  pd.concat([sna_micro_stats_a_df, sna_macro_stats_b_df, sociogram_micro_stats_df], axis=1)
        course_df = course_df.reset_index(names="id")
        course_df.insert(0, "corso", label)
        course_df = course_df.merge(associazione_df, left_on="id", right_on="soggetto")
        db = pd.concat([db, course_df])

db.to_csv("data_abgrid_per_analisi.xlsx", index=False)

                           soggeto lettera
0                    ALVERA' JARNO       A
1              BADITA MADALIN LUCA       B
2        BATTOCCHIO MARCELLO DANTE       C
3                  BECCALUVA SOFIA       D
4                   CASTAGNA GRETA       E
5            CECCARELLI ALESSANDRO       F
6          CHILLURA VITTORIA ELISA       G
7                 CICALINI ALESSIA       H
8          CIRIGLIANO DOMENICA PIA       I
9                  CUSCUSA MATILDE       J
10            D'ALESSANDRO ROBERTA       K
11                   D'ANNA GIULIA       L
12                   D'ELIA FLAVIO       M
13              D'ONOFRIO VIRGINIA       N
14                DEL BUONO ANDREA       O
15             DI GENNARO GIUSEPPE       P
16         DI LALLO MAXIM EMANUELE       Q
17                   DIELLA ANGELA       R
18             FRAPPARELLI DAMIANO       S
19  FRARE FRANCESCA MARIA VITTORIA       T
20              FUSCO MARIA TERESA       U
21              GASPERINI VIRGINIA       V
22         